In [436]:
import pandas as pd 
import numpy as np

In [437]:
properties = pd.read_csv("properties.csv", sep=",") 

In [438]:
neighbourhoods = pd.read_csv("neighbourhood.csv", sep=",")
neighbourhoods["neighbourhood"] = neighbourhoods["neighbourhood"].str.lower().str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf8")

Removendo anúncios

In [439]:
properties = properties.loc[properties["url"].dropna().index]
properties = properties.loc[properties["header"].dropna().index]

In [440]:
#properties["price"].unique()

In [441]:
len(properties)

9972

In [442]:
properties = properties.drop_duplicates(subset=["id", "header", "address", "area", "price"], 
                        keep="first")

In [443]:
properties = properties.reset_index(drop=False) 

In [444]:
properties["area"] = properties["area"].replace("--","0").astype(int)

In [445]:
properties["rooms"] = properties["rooms"].str.replace("--","0").str.split(" ").str[0].astype(int)

In [446]:
properties["price"] = properties["price"].str.split(" ").str[1].str.replace(".", "").astype(int)

#properties["price"] = properties["price"].str.split(" ").str[1].astype(float)

<ipython-input-446-e7f6d0900307>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  properties["price"] = properties["price"].str.split(" ").str[1].str.replace(".", "").astype(int)


In [447]:
properties["garages"] = properties["garages"].str.replace("--","0").str.split(" ").str[0].astype(int)

In [448]:
#properties["condo"] = properties["condo"].str.split(" ").str[1].replace(np.nan,"0").astype(float)
properties["condo"] = properties["condo"].str.split(" ").str[1].str.replace(".","").replace(np.nan,"0").astype(int)

<ipython-input-448-9ee6443d3e2c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  properties["condo"] = properties["condo"].str.split(" ").str[1].str.replace(".","").replace(np.nan,"0").astype(int)


In [449]:
properties["bathrooms"] = properties["bathrooms"].str.split(" ").str[0].replace("--","0").astype(int)

In [450]:
properties["url"] = properties["url"].astype(str)

In [451]:
properties["header"] = properties["header"].astype(str)

In [452]:
properties = pd.concat([properties, properties["amenities"].str.get_dummies(sep="|")], axis=1)

In [453]:
properties = properties.drop(columns=["amenities"])

In [454]:
properties["crawled_at"] = pd.to_datetime(properties["crawled_at"], format="%Y-%m-%d %H:%M:%S")

In [480]:
for idx,address in enumerate(properties["address"].str.lower().str.normalize("NFKD").str.encode("ascii", errors="ignore").str.decode("utf8")):
    for index,row in neighbourhoods.iterrows():
        if address.find(row["neighbourhood"]) != -1:
            if row["check_name"]:
                properties.loc[idx,"neighbourhood"] = row["equivalent"]
            else:
                properties.loc[idx,"neighbourhood"] = row["neighbourhood"]
            break

9896


In [488]:
len(properties)

9896